# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

API key looks good so far


In [3]:
# set up environment
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [6]:
# Get gpt-4o-mini to answer, with streaming

system_prompt=""" You are a coding assistant who answers user questions on Python programming language. """
stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
      ],
    stream=True
)
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

**Code Explanation**

The given code is written in Python and utilizes several advanced features, including generators and the `yield from` keyword.

python
yield from {book.get("author") for book in books if book.get("author")}


Let's break it down:

- `{...}`: This is a dictionary comprehension, which creates a new dictionary containing key-value pairs.
- `for book in books`: This is an iteration over the `books` collection. The value of each `book` object is used to compute some value (in this case, `"author"`).
- `if book.get("author")`: This filters out any `book` that doesn't have a key called `"author"`.
- `yield from ...`: This is where the magic happens. Instead of creating an explicit list or dictionary, it uses the values computed by the dictionary comprehension and yields them one by one.

**What does it do?**

In essence, this code creates a generator expression that:

1. Iterates over the `books` collection.
2. For each book, checks if it has an `"author"` key.
3. If it does, includes the value of that key in the output (i.e., the author's name).
4. Yields one author's name at a time.

The result is not stored in memory but generated on-the-fly as it's needed.

**Example Use Case**

Here's an example where this code might be used:

python
books = [
    {"title": "Book A", "author": "John Smith"},
    {"title": "Book B", "publisher": "ABC Press"},
    {"title": "Book C", "author": "Jane Doe"}
]

authors = yield from {book.get("author") for book in books if book.get("author")}
for author in authors:
    print(author)


This will output: `John Smith` and `Jane Doe`, but not the publisher's name since that value is not present in any of the books.

**Why use this code?**

Using a generator expression like this offers several benefits:

- **Memory Efficiency**: It doesn't require storing all authors' names in memory at once.
- **Lazy Evaluation**: The computation only happens when needed (i.e., when iterating over the output).
- **Concise Code**: It's written in a single line, making it easy to read and understand.

In [15]:
# Get Llama 3.2 to answer
import ollama
response = ollama.chat(model=MODEL_LLAMA, 
                       messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
      ],)
response = response['message']['content'].replace("```","").replace("markdown", "")
Markdown(response)
#update_display(Markdown(response), display_id=display_handle.display_id)

**Code Explanation**

The given code snippet is written in Python and utilizes the `yield from` keyword, which was introduced in Python 3.3.

Here's a breakdown of what this code does:

python
yield from {book.get("author") for book in books if book.get("author")}


*   `for book in books`: This loop iterates over each item in the `books` collection.
*   `if book.get("author")`: Inside the loop, it checks if the current book object has an "author" key. The `.get()` method is used here to avoid a KeyError in case the key does not exist.
*   `{book.get("author")}`: This dictionary comprehension extracts the author's name from each book that passes the previous condition and creates a new dictionary with just the author's name.

**The `yield from` Keyword**

Now, let's talk about `yield from`. When you use `yield from`, it allows you to "delegate" iteration over another iterable (in this case, the dictionary comprehension) without having to explicitly yield each item one by one. Instead of returning a single value, `yield from` returns an iterator that yields values from the delegated iterable.

In essence, `yield from {book.get("author") for book in books if book.get("author")}` can be seen as saying: "For each book that passes the condition, yield its author's name. This will create an iterator that produces these author names one by one."

**Why Use `yield from`?**

There are several reasons why someone might prefer using `yield from`:

*   **Efficiency**: Delegating iteration can be more memory-efficient than creating a list or other data structure to store the results.
*   **Flexibility**: By yielding each item individually, you have more control over how these items are processed. This is especially useful in generator functions, where the function itself becomes an iterator.

However, `yield from` might also make your code harder to understand, as it changes the way iteration works in that part of your code.

**Example Usage**

Here's a simple example to demonstrate this:

python
def author_names(books):
    return yield from {book.get("author") for book in books if book.get("author")}

# Define some sample data
books = [
    {"id": 1, "title": "Book A", "author": "John Doe"},
    {"id": 2, "title": "Book B", "author": None},
    {"id": 3, "title": "Book C", "author": "Jane Doe"}
]

# Create an iterator
for author in author_names(books):
    print(author)  # John Doe, Jane Doe


In summary, the given code snippet uses `yield from` to create an iterator that yields the authors' names of books that have authors. It is a more memory-efficient and flexible way to process these authors compared to using explicit for loops or other methods.